In [13]:
!pip install -q sentence-transformers
!pip install -q kaggle
!pip install -q nltk
!pip install -q pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [41]:
from google.colab import files

files.upload()
!mkdir ~/.kaggle

!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [28]:
import re

from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import nltk

patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
nltk.download('stopwords')
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc, n=5):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    word_counter = {}
    top = ''
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            if token in word_counter:
              word_counter[token] += 1
            else:
              word_counter[token] = 1
        popular_words = sorted(word_counter, key = word_counter.get, reverse = True)
        top = popular_words[:n]
    return " ".join(top)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
! kaggle datasets download -d 'dolfik/russian-telegram-chats-history'

100% 3.04G/3.04G [00:38<00:00, 108MB/s]
100% 3.04G/3.04G [00:38<00:00, 85.5MB/s]


In [24]:
!unzip russian-telegram-chats-history.zip

Archive:  russian-telegram-chats-history.zip
  inflating: telegram/Auff_02.json   
  inflating: telegram/BashkortostanR.json  
  inflating: telegram/BinanceRussian.json  
  inflating: telegram/Callisto_Russian.json  
  inflating: telegram/Casual_Chat.json  
  inflating: telegram/ChatDEEP.json  
  inflating: telegram/Geopolitica.json  
  inflating: telegram/GrupChatt.json  
  inflating: telegram/IslandOfHope.json  
  inflating: telegram/IvChat.json    
  inflating: telegram/JoyReactr.json  
  inflating: telegram/Kavardak20i.json  
  inflating: telegram/MAFIOZNIK_STUD.json  
  inflating: telegram/NeLiOne.json   
  inflating: telegram/PeaceDoBall.json  
  inflating: telegram/Penzachat.json  
  inflating: telegram/Poehavshie.json  
  inflating: telegram/PoshumimClub.json  
  inflating: telegram/ProGrow_chat.json  
  inflating: telegram/TalkChat.json  
  inflating: telegram/ToBeOrChat.json  
  inflating: telegram/TogliattiChat.json  
  inflating: telegram/TomskChat.json  
  inflating: teleg

In [40]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import time
import json
from tqdm import tqdm


# Model for computing sentence embeddings. We use one trained for similar questions detection
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')


user_dict = {}
with open('/content/telegram/blagoveshensk_chat.json', 'r') as chat:
  chat_log = json.load(chat)
  for message in tqdm(chat_log):
    if str(message["user_id"]) in user_dict:
      user_dict[str(message["user_id"])] += " " + message["text"] + "."
    else:
      user_dict[str(message["user_id"])] = message["text"]


(users,texts) = zip(*user_dict.items())
texts_grams = []
for text in tqdm(texts):
  texts_grams.append(lemmatize(text))


100%|██████████| 1272/1272 [26:10<00:00,  1.23s/it]


In [42]:
texts_grams[0]

'привет весь админ а ты'

In [43]:
corpus_embeddings = model.encode(texts_grams, batch_size=256, show_progress_bar=True, convert_to_tensor=True)

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [50]:
print("Start clustering")
start_time = time.time()

#Two parameters to tune:
#min_cluster_size: Only consider cluster that have at least 25 elements
#threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = util.community_detection(corpus_embeddings, min_community_size=25, threshold=0.75)

print("Clustering done after {:.2f} sec".format(time.time() - start_time))

#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:1]:
        print("\t", texts_grams[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-1:]:
        print("\t", texts_grams[sentence_id])
    print("\tUsers")
    for sentence_id in cluster[0:10]:
        print("\t", users[sentence_id])

Start clustering
Clustering done after 11.35 sec

Cluster 1, #555 Elements 
	 да а это ну я
	 ...
	 весь привет
	Users
	 354583049
	 876014
	 316034253
	 301014168
	 329489229
	 365574113
	 312563413
	 294871495
	 303842205
	 367326854

Cluster 2, #57 Elements 
	 вы привлечение 😁 цель нужно
	 ...
	 благовещенск привет весь приём рогов
	Users
	 375540526
	 356878830
	 344035377
	 270837716
	 221834988
	 156496935
	 386334041
	 301798039
	 430604691
	 468537109

Cluster 3, #29 Elements 
	 привет весь админ а ты
	 ...
	 я не это весь привет
	Users
	 354264637
	 248399606
	 353866461
	 272672459
	 171976124
	 229730614
	 155364575
	 356070492
	 396065812
	 449376097

Cluster 4, #28 Elements 
	 привет добрый от я спать
	 ...
	 добрый утро 🤟
	Users
	 222135783
	 341802488
	 433894913
	 272197462
	 222447031
	 472433319
	 101467050
	 471497010
	 477803448
	 501649987

Cluster 5, #28 Elements 
	 ну царь я а да
	 ...
	 дратуть
	Users
	 335677279
	 447153973
	 400804171
	 460529605
	 428334999
	

In [1]:
for i, cluster in enumerate(clusters):
    if i != 6:
      continue
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster:
        print("\t", texts_grams[sentence_id].split(" ")[0:100])
    break

NameError: ignored

In [ ]:
len(clusters)

23